# **Model Server**

_______________________________________________________________________________

<a id="top"></a>
## **steps**
**[notebook installs](#installs)**<br>
**[nuclio code section](#nuclio)**<br>
    - [inference server](#server)<br>
**[deploy](#deploy)**<br>
**[test deployment](#test)**<br>
**[test saved model object](#infderence)**


<a id="installs"></a>
_______________________________________________________________________________

## **notebook installs**

These are the packages we'll need to run this notebook, please install them once:

    !pip install -U kfserving==0.2.0 tensorflow==2.0.0b1 keras pandas 
    !pip install -U kubernetes==9.0.0
    !pip install -U azure

    !pip install -U git+https://github.com/yjb-ds/functions-demo.git
    !pip install -U git+https://github.com/mlrun/mlrun.git@development

<a id="nuclio"></a>
_______________________________________________________________________________

## **nuclio code section**

If you have never run nuclio functions in your notebooks, please uncomment and run the following:

    !pip install nuclio-jupyter

In [1]:
# nuclio: ignore
import nuclio

Install the following packages so they are available to the function:

In [2]:
%%nuclio cmd -c
pip install kfserving==0.2.0
pip install  numpy==1.16.4 tensorflow==2.0.0b1
pip install -U azure joblib
pip install -U git+https://github.com/mlrun/mlrun.git@development

In [3]:
%nuclio cmd pip uninstall -y functions
%nuclio cmd pip install -U git+https://github.com/yjb-ds/functions-demo.git

Uninstalling functions-0.1.0:
  Successfully uninstalled functions-0.1.0
  Cloning https://github.com/yjb-ds/functions-demo.git to /tmp/pip-req-build-2t1_xkyu
  Running setup.py bdist_wheel for functions ... done
  Stored in directory: /tmp/pip-ephem-wheel-cache-r3q7303o/wheels/2b/c7/ec/2c6ddbe3a11b59aa979259c6f2a12f1c7b8203c9e4c844ebe5
Successfully built functions
  Found existing installation: functions 0.1.0
    Can't uninstall 'functions'. No files were found to uninstall.


In [4]:
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)

import kfserving
import os
import numpy as np
import joblib
from typing import List
from tensorflow.keras.models import Sequential

from functions.models import FeaturesEngineer, classifier_gen
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline, Pipeline

class MyKerasClassifier(kfserving.KFModel):
    def __init__(self,
                 name: str,
                 model_dir: str,
                 classifier: Sequential = None):
        """TFKerasClassifier
        
        KubeFlow serving model wrapper.
        
        :param name:            model name
        :param model_dir: path of stored model
        :param classifier:      class type of classifier model
        
        """
        super().__init__(name)
        self.name = name
        self.model_dir = model_dir
        if classifier:
            self.ready = True

    def load(self):
        """Load model from KubeFlow storage.
        """
        ffg = FeaturesEngineer()

        # scaler
        ss = StandardScaler()
        ss.__dict__ = joblib.load(f"{self.model_dir}/scaler.pickle")

        # keras model
        ksm = classifier_gen()
        ksm.load_weights(f"{self.model_dir}/weights.h5")

        pipe = make_pipeline(ffg, ss, ksm)

        self.classifier = pipe

    def predict(self, body: List) -> List:
        """Generate model predictions from sample.
        
        :param body: A list of observations, each of which is an 1-dimensional feature vector.
            
        Returns model predictions as a `List`, one for each row in the `body` input `List`.
        """
        try:
            feats = np.asarray(body)
            result: np.ndarray = self.classifier.predict(feats)
            return result.tolist()
        except Exception as e:
            raise Exception(f"Failed to predict {e}")

In [5]:
# nuclio: end-code

<a id="deploy"></a>
_______________________________________________________________________________

## **deploy**

In [6]:
from mlrun import new_model_server, mount_v3io

In [7]:
TARGET_PATH = "/User/mlrun/simdata"
MODEL_NAME = "tfkeras_serving_v2"

In [8]:
fn = new_model_server(MODEL_NAME, 
                      models={"tfkeras_joblib_v2": TARGET_PATH}, 
                      model_class="MyKerasClassifier").apply(mount_v3io())

In [9]:
fn.to_dict()

{'kind': 'remote',
 'metadata': {'name': 'tfkeras-serving-v2', 'project': 'default'},
 'spec': {'command': '',
  'args': [],
  'image': '',
  'description': '',
  'volumes': [{'flexVolume': {'driver': 'v3io/fuse',
     'options': {'accessKey': '8d4c7612-c49a-471b-9e3c-9c02dbad7e76',
      'container': 'users',
      'subPath': '/admin'}},
    'name': 'v3io'}],
  'volume_mounts': [{'mountPath': '/User', 'name': 'v3io'}],
  'env': [{'name': 'SERVING_MODEL_tfkeras_joblib_v2',
    'value': '/User/mlrun/simdata'},
   {'name': 'TRANSPORT_PROTOCOL', 'value': 'seldon'},
   {'name': 'ENABLE_EXPLAINER', 'value': 'False'},
   {'name': 'MODEL_CLASS', 'value': 'MyKerasClassifier'},
   {'name': 'V3IO_API', 'value': 'v3io-webapi.default-tenant.svc:8081'},
   {'name': 'V3IO_USERNAME', 'value': 'admin'},
   {'name': 'V3IO_ACCESS_KEY',
    'value': '8d4c7612-c49a-471b-9e3c-9c02dbad7e76'}],
  'config': {'spec.triggers.http': {'kind': 'http',
    'maxWorkers': 8,
    'attributes': {'ingresses': {}},
    '

In [10]:
#fn.verbose=True
addr = fn.deploy(project="refactoring-demos")

[mlrun] 2019-12-31 21:50:47,473 deploy started
[nuclio] 2019-12-31 21:53:19,842 (info) Build complete
[nuclio] 2019-12-31 21:53:26,926 done updating tfkeras-serving-v2, function address: 3.137.70.243:31757


## **test**

In [11]:
import pyarrow.parquet as pq
import pyarrow as pa
import pandas as pd

import requests

In [12]:
# Grab some data - balanced dataset
features = pd.read_csv("x_test_50.csv")
labels = pd.read_csv("y_test_50.csv")

In [13]:
# Seldon protocol event
event_seldon = {
    "data" : {
        "ndarray": features.values
    }
}
csel = str(event_seldon).replace("\"", "\"").replace("\n", "").replace(" ", "")

In [14]:
# resp = requests.put(addr + "/predict/tfkeras_joblib_v2", data=csel)
# print(resp.content)

In [15]:
# ?????????

In [16]:
resp = requests.put(addr + "/tfkeras_joblib_v2/predict", data=csel)
print(resp.content)

b'Exception caught in handler "Unrecognized request format: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)": Traceback (most recent call last):\n  File "/usr/local/lib/python3.6/site-packages/mlrun/runtimes/serving.py", line 96, in parse_event\n    body = json.loads(event.body)\n  File "/usr/local/lib/python3.6/json/__init__.py", line 354, in loads\n    return _default_decoder.decode(s)\n  File "/usr/local/lib/python3.6/json/decoder.py", line 339, in decode\n    obj, end = self.raw_decode(s, idx=_w(s, 0).end())\n  File "/usr/local/lib/python3.6/json/decoder.py", line 355, in raw_decode\n    obj, end = self.scan_once(s, idx)\njson.decoder.JSONDecodeError: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)\n\nDuring handling of the above exception, another exception occurred:\n\nTraceback (most recent call last):\n  File "/opt/nuclio/_nuclio_wrapper.py", line 176, in serve_requests\n    entrypoint_output = self._entrypoint(self._conte